## This file includes all the functions and code we developed / used so far

In [ ]:
# For sending GET requests from the API
import requests
# For saving access tokens and for file management when creating and adding to the dataset
import os
# For dealing with json responses we receive from the API
import json
# For displaying the data after
import pandas as pd
# For saving the response data in CSV format
import csv
# For parsing the dates received from twitter in readable formats
import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time, datetime
import tweeterid

# Helper class to convert a DynamoDB item to JSON.
class DecimalEncoder(json.JSONEncoder):
    def default(self, o):
        if isinstance(o, Decimal):
            if o % 1 > 0:
                return float(o)
            else:
                return int(o)
        return super(DecimalEncoder, self).default(o)
    
# academic token - the token of IDC
os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'

In [4]:
### General function that are used in many functions we built:

def auth():
    return os.getenv('TOKEN')

def create_headers(bearer_token):
    headers = {"Authorization": "Bearer {}".format(bearer_token)}
    return headers

def connect_to_endpoint(url, headers, params, next_token = None, verbose = True):
    params['next_token'] = next_token   #params object received from create_url function
    response = requests.request("GET", url, headers = headers, params = params)
    if verbose == True:
        print("Endpoint Response Code: " + str(response.status_code))
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


## cool functions

### cool function 1 

Given a tweet-id, get a url for the tweet in twitter so you can watch it yourseld (and see its comments, likes, etc)

In [ ]:
def get_url_by_tweet_id(tweet_id):
    url_tweet = "https://twitter.com/anyuser/status/" + str(tweet_id)
    print(url_tweet)
    return url_tweet

In [18]:
get_url_by_tweet_id("1466035318365294603")

https://twitter.com/anyuser/status/1466035318365294603


'https://twitter.com/anyuser/status/1466035318365294603'

# Function 1 - GET tweet by tweetid:

This function gets as input:
1. search url - the endpoint (which is basically the same so we can just insert it t the function)
2. tweet-id

It gets a tweet id and returns the tweet itself and info about the author of the tweet.

In [5]:
def create_url_tweet_id(search_url, tweet_id):
    
    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
    search_url = search_url.replace(":id", tweet_id)
    #change params based on the endpoint you are using
    query_params = {
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)


########################################################### use:
#search_url, query_params = create_url_tweet_id(search_url, tweet_id)
#connect_to_endpoint(url, headers, params)

In [7]:
#search url for idtweet - App rate limit: 300 requests per 15-minute window shared among all users of your app
#User rate limit (OAuth 1.0a): 900 requests per 15-minute window per each authenticated user
headers = create_headers(os.environ['TOKEN'])
search_url = "https://api.twitter.com/2/tweets/:id" #Change to the endpoint you want to collect data from
tweet_id = "1470753200311517189" #"1108157488581562373"
###-------------------------------------------------------------------------------------
search_url, query_params = create_url_tweet_id(search_url, tweet_id)
json_response = connect_to_endpoint(search_url, headers, query_params) #new

Endpoint Response Code: 200


### here we can see in a nicer way the data from the tweet - need to handle cases where some of the fields are missing in the json (otherwise we will get an error)

In [ ]:
print("*********** Data on the tweet: ***************")
# print("source:", json_response['data']["source"])
# print("conversation_id:", json_response['data']["conversation_id"])
# print("id:", json_response['data']["id"])
# print("Tweet url:", "https://twitter.com/roy/status/"+json_response['data']["id"])
# print("lang:", json_response['data']["lang"])
# print("created_at:", json_response['data']["created_at"])

# print("retweet_count:", json_response['data']["public_metrics"]["retweet_count"])
# print("like_count:", json_response['data']["public_metrics"]["like_count"])
# print("reply_count:", json_response['data']["public_metrics"]["reply_count"])
# print("quote_count:", json_response['data']["public_metrics"]["quote_count"])

# #print("referenced_tweets:", json_response['data']["referenced_tweets"])
# print("author_id:", json_response['data']["author_id"])
# print("reply_settings:", json_response['data']["reply_settings"])
# print()
# print("*********** The tweet: ***************")
# print("text:", json_response['data']["text"])
# print()

# print("*********** Data on the user: ***************")
# print("Verified:", json_response['includes']["users"][0]["verified"])
# print("name:", json_response['includes']["users"][0]["name"])
# print("username:", json_response['includes']["users"][0]["username"])
# print("followers_count:", json_response['includes']["users"][0]["public_metrics"]["followers_count"])
# print("following_count:", json_response['includes']["users"][0]["public_metrics"]["following_count"])
# print("tweet_count:", json_response['includes']["users"][0]["public_metrics"]["tweet_count"])
# print("listed_count:", json_response['includes']["users"][0]["public_metrics"]["listed_count"])


# Function 2 - GET tweets by keyword and timeframes:

We need to organize the code in a function

In [9]:
def create_url(search_url, keyword, start_date, end_date, max_results = 10):
    
    
    #search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from

    #change params based on the endpoint you are using
    query_params = {'query': keyword,
                    'start_time': start_date,
                    'end_time': end_date,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {}}
    return (search_url, query_params)
# --------------------------------------------------------------------------------------------------------------
#search url for recent tweets - recent 7 days
search_url = "https://api.twitter.com/2/tweets/search/recent" #Change to the endpoint you want to collect data from
keyword = "fun"
start_time ="2021-11-29T00:00:00.000Z"
end_time = "2021-12-01T00:00:00.000Z"
max_results = 10 #minimum 10 and maximum 100 !
###-------------------------------------------------------------------------------------
url = create_url(search_url, keyword, start_time,end_time, max_results)
json_response = connect_to_endpoint(url[0], headers, url[1])

Endpoint Response Code: 400


Exception: (400, '{"errors":[{"parameters":{"start_time":["2021-11-29T00:00Z"]},"message":"Invalid \'start_time\':\'2021-11-29T00:00Z\'. \'start_time\' must be on or after 2022-03-03T17:09Z"},{"parameters":{"end_time":["2021-12-01T00:00Z"]},"message":"Invalid \'end_time\':\'2021-12-01T00:00Z\'. \'end_time\' must be on or after 2022-03-03T17:09Z"}],"title":"Invalid Request","detail":"One or more parameters to your request was invalid.","type":"https://api.twitter.com/2/problems/invalid-request"}')

# Function 2 - GET tweets by keyword and timeframes:

We need to organize the code in a function
## Function return_tweets_of_key_opinion_leader:
The function retrives tweets written by a key opinion leader

+ **query** = The query to use for seach tweets, you can insert an hashtag or any other words to filter the tweets, by default the function doesn't use any such filter. **Nite** you can write the whole query (including the name of the user to search for his tweets) in this argument. if you choose to do so, don't change the **`user_name`** argument.

#### how to create the query - how to select the user to search for and keywords to search:
https://developer.twitter.com/en/docs/twitter-api/tweets/counts/integrate/build-a-query

+ **user_name** = The name of the key opinion leader to search for his tweets

+ **start_time** = The start date to search tweets from -> format = "2015-12-7T00:00:00Z" 

+ **end_time** = The end date to search tweets from -> format = "2021-12-26T00:00:00Z" 

+ **max_results** = The maximum number of search results to be returned by a request. A number between 10 and the system limit (currently 500). By default, a request response will return 10 results.

+ number_of_desired_tweets_to_return = The number of tweets, written by the user, that we wish to get back, if there are less than this number - its stil fine

In [145]:
def return_tweets_of_key_opinion_leader(query="", user_name=None,
                                        start_time = "2015-12-7T00:00:00Z",
                                        end_time = "2021-12-26T00:00:00Z",
                                        max_results = 10, evaluate_last_token = False,
                                        limit_amount_of_returned_tweets = 10000000,
                                       verbose_10 = False):
    import os
    try:
        os.environ['TOKEN'] = 'AAAAAAAAAAAAAAAAAAAAADRFOwEAAAAAaTp%2Bdd1OhobYMYb5ExPXm7IL6RA%3DDHknH402gOXoegUGxNtpC6giIjdackRLtdRx6tjrnnLeFN1ntT'
        headers = create_headers(os.environ['TOKEN'])
    except:
        print("You must define a token")
        print("try someting like: os.environ['TOKEN'] = 'XXXXXXXXX'")
        
    def connect_to_endpoint(url, headers, params, next_token = None, verbose = True):
        params['next_token'] = next_token   #params object received from create_url function
        response = requests.request("GET", url, headers = headers, params = params)
        if verbose == True:
            print("Endpoint Response Code: " + str(response.status_code))
        if response.status_code == 200:
            return response.json()
        if response.status_code == 429:
            print(response.text)
            print("try sleeping for 2 seconds")
            time.sleep(2)
            return response.status_code
            #print("need to sleep for 15 minutes")
            #time.sleep(15*60)
        elif response.status_code != 200:
            raise Exception(response.status_code, response.text)
        else:
            return response.json()

    
    search_url = "https://api.twitter.com/2/tweets/search/all" #endpoint use to collect data from


    if user_name is not None:
        query = str(query) + " from:" + str(user_name)
        try:
            display_start_time = datetime.strptime(start_time.split("T")[0], "%Y-%m-%d").strftime("%d-%m-%Y")
            display_end_time = datetime.strptime(end_time.split("T")[0], "%Y-%m-%d").strftime("%d-%m-%Y")
        except:
            display_start_time = start_time
            display_end_time = end_time
        print("Bringing all the tweets of the user:", user_name, "from:", display_start_time, "to", display_end_time)
        print()

    ##### the log dir
    import os.path
    dir_name = "key_opinion_leaders_tweets_tables_beta"
    dir_name_beta = "key_opinion_leaders_tweets_tables_beta"
    dir_log_name = os.path.join(dir_name_beta, "log_key_opinion_leaders") 

    try:
        os.mkdir(dir_name)
        print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
    except:
        print("The dir", dir_name ,"already exist")
    try:
        os.mkdir(dir_log_name)
        print("creating directory", dir_log_name, "to insert all the logs of the key opinion leaders")
    except:
        print("The dir", dir_log_name ,"already exist")
    ########################
    path_for_log_dir_of_certain_user = os.path.join(dir_log_name, user_name)
    try:
        os.mkdir(path_for_log_dir_of_certain_user)
        print("creating directory", path_for_log_dir_of_certain_user,"in the dir",dir_log_name, "to insert all the logs of the key opinion leader", user_name)
    except:
        print("The dir", path_for_log_dir_of_certain_user ,"already exist")
        
    path_for_dir_retriving_tweets_streem = os.path.join(path_for_log_dir_of_certain_user, 'retriving_tweets_streem.txt')
    with open(path_for_dir_retriving_tweets_streem, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time)
        f.write('\n')
        f.write('\n')
        
    ########### If the token file exist already, then take the last token available, else start from token 1  ############ 
    tokens_location = os.path.join(dir_log_name, user_name, "tokens.txt") 

    if (evaluate_last_token == True and os.path.isfile(tokens_location) == True):
        a_file = open(tokens_location, "r")
        lines = a_file.readlines()
        last_lines = lines[-2]
        next_token = last_lines[0:-1]
        a_file.close()    
    else:
        next_token = None
        
    ################ Add a time stamp ########################################
    path_for_dir_tokens = os.path.join(path_for_log_dir_of_certain_user, 'tokens.txt')
    with open(path_for_dir_tokens, 'a') as f:
        from datetime import datetime
        now = datetime.now()
        current_time = now.strftime("%H:%M:%S (Date: %d.%m.%y)")
        current_time = "Current Time: " + current_time + "   *****************************************   "
        f.write(current_time)
        f.write('\n')
        f.write('\n')
        
    ##########################################################################################

    continue_searching = True
    json_response_list = []
    next_tokens = []
    num_of_returned_tweets = 0
    counter_loops = 0
    
    while continue_searching == True and num_of_returned_tweets < limit_amount_of_returned_tweets:
        counter_loops +=1
        if counter_loops > 1:
            next_token = json_response["meta"]["next_token"]
            query_params["next_token"] = next_token
            print("token to insert:",next_token)
        #if the returned amount of tweets is getting close to the limit number, we need to alter the max_result,
        #so we won't get tweets beyond what we asked
        
        if (limit_amount_of_returned_tweets - num_of_returned_tweets) < max_results:
            max_results = limit_amount_of_returned_tweets - num_of_returned_tweets
            if max_results < 10:
                max_results = 10
        else :
            max_results = max_results

        #change params based on the endpoint you are using
        query_params = {'query': query,
                    'start_time': start_time,
                    'end_time': end_time,
                    'max_results': max_results,
                    'expansions': 'author_id,in_reply_to_user_id,geo.place_id,entities.mentions.username,referenced_tweets.id',
                    'user.fields': 'id,name,username,created_at,description,public_metrics,verified',
                    'tweet.fields': 'id,text,author_id,in_reply_to_user_id,geo,conversation_id,created_at,lang,public_metrics,referenced_tweets,reply_settings,source',
                    'place.fields': 'full_name,id,country,country_code,geo,name,place_type',
                    'next_token': {next_token}}
        
        json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
        
        if json_response == 429:
            json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
            if json_response == 429:
                print("sleep 10 seconds") #need to sleep for 15 minutes")
                time.sleep(10)
                json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
                if json_response == 429:
                    print("sleep 30 seconds") #need to sleep for 15 minutes")
                    time.sleep(30)
                    json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
                    if json_response == 429:
                        print("sleep 60 seconds") #need to sleep for 15 minutes")
                        time.sleep(60)
                        json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
                        if json_response == 429:
                            print("sleep 15 Minutes") #need to sleep for 15 minutes")
                            time.sleep(15*60)
                            json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)
                            if json_response == 429:
                                print("sleep 15 Minutes") #need to sleep for 15 minutes")
                                time.sleep(15*60)
                                json_response = connect_to_endpoint(search_url, headers, query_params, next_token = next_token, verbose = False)

        json_response_list.append(json_response) #the first json_response itme
        num_of_returned_tweets += json_response["meta"]["result_count"]

        ##### making a dataframe out of the json response:
        try:
            a = pd.json_normalize(json_response["data"])
            b = pd.json_normalize(json_response["includes"], ["users"]).add_prefix("users.")
            
            a.conversation_id = a.conversation_id.astype("string")
            a.id = a.id.astype("string")
            a["id_new"] = "id: " + a["id"].astype("string")
            a["conv_id_new"] = "conv_id: " + a["conversation_id"].astype("string")
        #c = pd.json_normalize(json_response["includes"]["places"]).add_prefix("places.")
            df_tweets_i = pd.merge(a, b, left_on="author_id", right_on="users.id")
            list_of_cols_to_add = ['author_id', 'conversation_id', "conv_id_new", "id", "id_new",
                                   'created_at','entities.mentions',
                        'public_metrics.like_count', 'public_metrics.quote_count',
                       'public_metrics.reply_count', 'public_metrics.retweet_count','referenced_tweets', 'text',
                        'users.created_at', 'users.description','users.id', 'users.name',
                       'users.public_metrics.followers_count', 'users.public_metrics.following_count',
                       'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
                       'users.username', 'users.verified']

            list_cols_to_drop = [x for x in df_tweets_i.columns if x not in list_of_cols_to_add]

            ##droping labels we don't need
            df_tweets_i = df_tweets_i.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

            for col in list_of_cols_to_add:
                if col in df_tweets_i.columns:
                    pass
                else:
                    df_tweets_i[col] = "NA"

            #sort columns by alphabetic order
            col_list_df_tweets_i = df_tweets_i.columns.tolist()
            col_list_df_tweets_i.sort()
            df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)
            
            name = user_name + ".csv"
            path_for_table = os.path.join(dir_name_beta, name)
            if os.path.isfile(path_for_table) == False: #if this is the first table of tweets
                df_tweets_i.to_csv(path_for_table, index=True)
            else:
                df_tweets_i.to_csv(path_for_table, mode='a', index=True, header=False)
        except:
            print("no data / include in the json")
       
        with open(path_for_dir_retriving_tweets_streem, 'a') as f:
            print_stat = str(counter_loops) + " -> Got from twitter " + str(json_response["meta"]["result_count"]) + " tweets, and there are more tweets of that user to get, I am bringing more tweets!"
            f.write(print_stat)
            f.write('\n')
            print_total = "Total amount of tweets: " + str(num_of_returned_tweets)
            f.write(print_total)
            f.write('\n')
            f.write('\n')

        if "next_token" in json_response["meta"]:
            if (verbose_10 == True and counter_loops % 20 == 1):
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!")
                print()
            elif verbose_10 == False:
                print(counter_loops, "Got from twitter", json_response["meta"]["result_count"], "tweets, and there are more tweets of that user to get, I am bringing more tweets!")
                print()
            next_token = json_response["meta"]["next_token"]
            query_params["next_token"] = next_token
            next_tokens.append(next_token)
            #ids_token_print = "next token = " + next_token + "newest id: " + json_response["meta"]["newest_id"] + " | oldest id: " + json_response["meta"]["oldest_id"]
            ids_token_print = next_token
            with open(path_for_dir_tokens, 'a') as f:
                f.write(ids_token_print)
                f.write('\n')
                f.write('\n')
        else:
            print("no more tweets from this user")
            continue_searching = False
            print("Total amount of collected tweets = ", num_of_returned_tweets)
        
        if num_of_returned_tweets >=limit_amount_of_returned_tweets:
            print("oooops, There may be more tweets to return, but you asked to limit the amount of returned tweets")
            print("infact you got", num_of_returned_tweets, "returned tweets and limited the function to get", limit_amount_of_returned_tweets, "tweets")
    
    #In what case we suspect that there may be more tweets that we didn't get? -->
    #When the number of tweets we asked to get is equal to the number of tweets we got back
    
    ### save all thr json responses in json file:
    path_for_dir_all_json_responses = os.path.join(path_for_log_dir_of_certain_user, 'all_json_responses.json')
    with open(path_for_dir_all_json_responses, 'w') as outfile:
        json.dump(json_response_list, outfile)
    return json_response_list, num_of_returned_tweets, next_tokens


## For loop to get all the tweets written by a list of key opinion leaders in a certain time interval

We are going to get a list with X elements (X = the number of key opinion leaders we insert to the function), each such element is also a list, this time this list includes all the tweets by this user in the given time interval.

In [20]:
print(tweeterid.handle_to_id('darrengrimes_'))
print(tweeterid.id_to_handle('4691437897'))

4691437897
@darrengrimes_


In [1]:
#making the data and log dir
dir_name = "key_opinion_leaders_tweets_tables_beta"
#dir_log_name = "log_key_opinion_leaders"
#path_for_log_dir = os.path.join(dir_name, dir_log_name)

import os.path
try:
    os.mkdir(dir_name)
    print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
except:
    print("The dir", dir_name ,"already exist")

creating directory key_opinion_leaders_tweets_tables_beta to insert all the tables of all the key opinion leaders


In [2]:
# all_key_opinion_leader_names = ["theresa_may","NicolaSturgeon","MichelBarnier","donaldtusk","fhollande","PhilipHammondUK","DavidDavisMP","JunckerEU",
# "guyverhofstadt","DidierSeeuws","EndaKennyTD","hilarybennmp","MinPres","MartinSelmayr","Keir_Starmer","realDonaldTrump",
# "Wolf_Schauble","LiamFox","TimmermansEU","BorisJohnson","Nigel_Farage","NJ_Timothy","DacrePaul","ManfredWeber",
# "davidmcallister","matteorenzi","PJWheatcroft","SadiqKhan","marianorajoy","HeywoodFndation","MartinSchulz","cbicarolyn",
# "rupertmurdoch","WeyandSabine","OllyRobbins","MLP_officiel","MarkJCarney","FrancesOGrady","giannipittella","MalmstromEU",
# "TheGinaMiller","CER_Grant","shriti_vadera","carlosghosn"]


In [3]:
all_key_opinion_leader_names = ["DavidDavisMP"]


In [4]:
#all_key_opinion_leader_names = ["DominicCumins", "darrengrimes_", "PhilipHammondUK","roymadpis789"]
#MakhoulRawaa
start_time = "2015-12-7T00:00:00Z"
end_time = "2021-12-04T00:00:00Z"
max_results = 500
limit_amount_of_returned_tweets = 1000000
############################################################################################
#### we don't want to get data on key opinion leaders that we already searched for their tweets!
# thus we first need to see which of the users that are inserted in the list are **not** in the files we already have:
from os import listdir
from os.path import isfile, join
mypath = "key_opinion_leaders_tweets_tables_beta"
users_we_have_data = [f.split(".")[0] for f in listdir(mypath) if isfile(join(mypath, f))]

key_opinion_leader_names = list(set(all_key_opinion_leader_names) - set(users_we_have_data))
print("Users to search their tweets:",key_opinion_leader_names)
print("-----------------------------------------------")
print("Users we already have data on:",list(set(all_key_opinion_leader_names).intersection(users_we_have_data)))



Users to search their tweets: ['DavidDavisMP']
-----------------------------------------------
Users we already have data on: []


I altered the function so it wil now save the data on each user in a csv and just after doing so it will move on to the next user

In [7]:

#making the data and log dir
dir_name = "key_opinion_leaders_tweets_tables_beta"
#dir_log_name = "log_key_opinion_leaders"
#path_for_log_dir = os.path.join(dir_name, dir_log_name)

import os.path
try:
    os.mkdir(dir_name)
    print("creating directory", dir_name, "to insert all the tables of all the key opinion leaders")
except:
    print("The dir", dir_name ,"already exist")

# try: 
#     os.mkdir(path_for_log_dir)
#     print("creating directory", path_for_log_dir, "in the directory",dir_name, "to insert all the log of all the key opinion leaders")
# except:
#     print("The dir", path_for_log_dir ,"already exist")
#########################################################################    
users_json_response_lists = []
names_evaluated = []
names_didnt_evaluated = []
next_tokens_users= [] #this will include a list where eachelement is a list containing all the tokens off the specific user
for name in key_opinion_leader_names:
    print("bringing tweets of", name)
    query = ""
    user_name = name
    try:
        #max_results = 500

        json_response_list, num_of_returned_tweets,next_tokens = return_tweets_of_key_opinion_leader(query=query, user_name=user_name,
                                                start_time = start_time,
                                                end_time = end_time,
                                                max_results = max_results,
                                                limit_amount_of_returned_tweets = limit_amount_of_returned_tweets,
                                                                                                     verbose_10 = True)
        print(num_of_returned_tweets)
        if num_of_returned_tweets > 0:
            names_evaluated.append(name)
            #users_json_response_lists.append(json_response_list)
            next_tokens_users.append(next_tokens)
            #df_all_tweets_user = unite_tweets_in_one_df(json_response_list)
            #name_file = user_name + ".csv"
            #path_for_table = os.path.join(dir_name, name_file)
            #print(path_for_table)
            #df_all_tweets_user.to_csv(path_for_table)

        else:
            names_didnt_evaluated.append(name)
            print("The user:", name, "had", num_of_returned_tweets, "tweets!!")

        print("---------------------------------------------------------------")
    except:
        print("There was a problem with the key opinion leader:", name)
        names_didnt_evaluated.append(name)
        print("*************************************************************************************")

The dir key_opinion_leaders_tweets_tables_beta already exist
bringing tweets of DavidDavisMP
There was a problem with the key opinion leader: DavidDavisMP
*************************************************************************************


In [34]:
# os.chdir("C:/Users/Roy/Documents/A IDC Roy BDA/Third Year - BDA/Semester 2/סטאז'/code new")


In [45]:
import json
os.getcwd()
# Opening JSON file
f = open('all_json_responses.json')
data = json.load(f)

# for i in data['emp_details']:
#     print(i)



In [ ]:
1466720116905857024

In [46]:
data

[{'data': [{'source': 'Twitter Web App',
    'conversation_id': '1466720116905857024',
    'created_at': '2021-12-03T10:45:00.000Z',
    'entities': {'mentions': [{'start': 3,
       'end': 13,
       'username': 'talkRADIO',
       'id': '3380282686'}]},
    'author_id': '2797521996',
    'text': 'RT @talkRADIO: Germany is to make Covid jabs mandatory.\n\nTory MP David Davis: "Nuremberg guidelines say you can\'t force someone to have a m…',
    'public_metrics': {'retweet_count': 2142,
     'reply_count': 0,
     'like_count': 0,
     'quote_count': 0},
    'id': '1466720116905857024',
    'reply_settings': 'everyone',
    'lang': 'en',
    'referenced_tweets': [{'type': 'retweeted', 'id': '1466710447877468160'}]},
   {'source': 'Twitter Web App',
    'conversation_id': '1466113276052451331',
    'created_at': '2021-12-01T18:33:38.000Z',
    'author_id': '2797521996',
    'text': "The Paxlovid treatment from Pfizer, alongside our vaccination effort, is game changing for the route out 

limit_output extension: Maximum message size of 10000 exceeded with 1532006 characters

In [54]:
data[0]["data"]

[{'source': 'Twitter Web App',
  'conversation_id': '1466720116905857024',
  'created_at': '2021-12-03T10:45:00.000Z',
  'entities': {'mentions': [{'start': 3,
     'end': 13,
     'username': 'talkRADIO',
     'id': '3380282686'}]},
  'author_id': '2797521996',
  'text': 'RT @talkRADIO: Germany is to make Covid jabs mandatory.\n\nTory MP David Davis: "Nuremberg guidelines say you can\'t force someone to have a m…',
  'public_metrics': {'retweet_count': 2142,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'id': '1466720116905857024',
  'reply_settings': 'everyone',
  'lang': 'en',
  'referenced_tweets': [{'type': 'retweeted', 'id': '1466710447877468160'}]},
 {'source': 'Twitter Web App',
  'conversation_id': '1466113276052451331',
  'created_at': '2021-12-01T18:33:38.000Z',
  'author_id': '2797521996',
  'text': "The Paxlovid treatment from Pfizer, alongside our vaccination effort, is game changing for the route out of this pandemic. \n\nI asked the PM what the UK is 

limit_output extension: Maximum message size of 10000 exceeded with 323804 characters

In [131]:
a = pd.json_normalize(data[0]["data"])


In [71]:
pd.to_numeric(a.conversation_id )

0      1466720116905857024
1      1466113276052451331
2      1466097290112872449
3      1464176482914390021
4      1464176482914390021
              ...         
495    1303441511686823936
496    1302982161310547979
497    1302595225915985928
498    1302552249919508480
499    1298922310750142464
Name: conversation_id, Length: 500, dtype: int64

In [72]:
# a.conversation_id = pd.to_numeric(a.conversation_id )
# a.id = pd.to_numeric(a.id )

In [143]:
a.conversation_id = a.conversation_id.astype("string")
a.id = a.id.astype("string")
a["id_new"] = "id: " + a["id"].astype("string")
a["conv_id_new"] = "conv_id: " + a["conversation_id"].astype("string")

In [136]:
b = pd.json_normalize(data[0]["includes"], ["users"]).add_prefix("users.")
b

,users.created_at,users.description,users.id,users.username,users.name,users.verified,users.public_metrics.followers_count,users.public_metrics.following_count,users.public_metrics.tweet_count,users.public_metrics.listed_count
0,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,DavidDavisMP,David Davis,True,188537,39,1742,1333
1,2015-07-17T11:48:34.000Z,The home of talkRADIO and talkRADIO TV.\ntalkT...,3380282686,talkRADIO,talkRADIO,True,198567,28,88980,746
2,2008-09-18T06:50:54.000Z,"Think ahead with the latest news, comment, ana...",16343974,Telegraph,The Telegraph,True,3206051,867,470953,20514
3,2011-03-21T13:07:52.000Z,"Chancellor of the Duchy of Lancaster, Chief of...",269788397,SteveBarclay,Steve Barclay,True,37161,748,3534,861
4,2010-04-05T22:13:47.000Z,"Member of Parliament for everyone in Stroud, t...",129951559,Siobhan_Baillie,Siobhan Baillie MP,True,12972,4349,5706,206
...,...,...,...,...,...,...,...,...,...,...
103,2009-01-21T15:16:49.000Z,We are a legal action NGO. We defend people su...,19290985,Reprieve,Reprieve,True,41827,1287,19219,738
104,2016-01-15T22:24:05.000Z,"MP for Esher and Walton, Deputy PM, Lord Chanc...",4764882552,DominicRaab,Dominic Raab,True,343317,344,5558,1753
105,2008-04-10T17:21:52.000Z,"We lead the UK’s work internationally, promoti...",14353202,FCDOGovUK,"Foreign, Commonwealth & Development Office",True,1040006,3106,44333,6322
106,2009-02-03T20:24:44.000Z,Labour MP for Birmingham Yardley\n\nhttps://t....,20000725,jessphillips,Jess Phillips MP,True,516257,3438,74061,2135


In [137]:
df_tweets_i = pd.merge(a, b, left_on="author_id", right_on="users.id")
list_of_cols_to_add = ['author_id', 'conversation_id', "id", "id_new",'created_at','entities.mentions',
    'public_metrics.like_count', 'public_metrics.quote_count',
   'public_metrics.reply_count', 'public_metrics.retweet_count','referenced_tweets', 'text',
    'users.created_at', 'users.description','users.id', 'users.name',
   'users.public_metrics.followers_count', 'users.public_metrics.following_count',
   'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
   'users.username', 'users.verified']

list_cols_to_drop = [x for x in df_tweets_i.columns if x not in list_of_cols_to_add]

##droping labels we don't need
df_tweets_i = df_tweets_i.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

In [138]:
df_tweets_i.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   conversation_id                       500 non-null    float64
 1   created_at                            500 non-null    object 
 2   author_id                             500 non-null    object 
 3   text                                  500 non-null    object 
 4   id                                    500 non-null    string 
 5   referenced_tweets                     209 non-null    object 
 6   entities.mentions                     138 non-null    object 
 7   public_metrics.retweet_count          500 non-null    int64  
 8   public_metrics.reply_count            500 non-null    int64  
 9   public_metrics.like_count             500 non-null    int64  
 10  public_metrics.quote_count            500 non-null    int64  
 11  id_new             

In [139]:
for col in list_of_cols_to_add:
    if col in df_tweets_i.columns:
        pass
    else:
        df_tweets_i[col] = "NA"

In [140]:
#sort columns by alphabetic order
col_list_df_tweets_i = df_tweets_i.columns.tolist()
col_list_df_tweets_i.sort()
df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)

In [141]:
df_tweets_i.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 500 entries, 0 to 499
Data columns (total 22 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   author_id                             500 non-null    object 
 1   conversation_id                       500 non-null    float64
 2   created_at                            500 non-null    object 
 3   entities.mentions                     138 non-null    object 
 4   id                                    500 non-null    string 
 5   id_new                                500 non-null    string 
 6   public_metrics.like_count             500 non-null    int64  
 7   public_metrics.quote_count            500 non-null    int64  
 8   public_metrics.reply_count            500 non-null    int64  
 9   public_metrics.retweet_count          500 non-null    int64  
 10  referenced_tweets                     209 non-null    object 
 11  text               

In [142]:
df_tweets_i.to_csv("roy1.csv", index=True)

In [116]:
df_tweets_i

,author_id,conversation_id,created_at,entities.mentions,id,public_metrics.like_count,public_metrics.quote_count,public_metrics.reply_count,public_metrics.retweet_count,referenced_tweets,...,users.created_at,users.description,users.id,users.name,users.public_metrics.followers_count,users.public_metrics.following_count,users.public_metrics.listed_count,users.public_metrics.tweet_count,users.username,users.verified
0,2797521996,1.466720e+18,2021-12-03T10:45:00.000Z,"[{'start': 3, 'end': 13, 'username': 'talkRADI...",1466720116905857024,0,0,0,2142,"[{'type': 'retweeted', 'id': '1466710447877468...",...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
1,2797521996,1.466113e+18,2021-12-01T18:33:38.000Z,NaN,1466113276052451331,66,6,45,10,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
2,2797521996,1.466097e+18,2021-12-01T17:30:07.000Z,"[{'start': 160, 'end': 170, 'username': 'Teleg...",1466097290112872449,38,3,27,8,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
3,2797521996,1.464176e+18,2021-11-26T11:10:43.000Z,NaN,1464189872156119040,24,0,3,1,"[{'type': 'replied_to', 'id': '146417648449974...",...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
4,2797521996,1.464176e+18,2021-11-26T10:17:31.000Z,NaN,1464176484499742723,17,0,2,0,"[{'type': 'replied_to', 'id': '146417648291439...",...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,2797521996,1.303442e+18,2020-09-08T21:14:07.000Z,NaN,1303441511686823936,1005,24,76,311,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
496,2797521996,1.302982e+18,2020-09-07T14:48:50.000Z,"[{'start': 15, 'end': 27, 'username': 'grantsh...",1302982161310547979,82,0,20,7,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
497,2797521996,1.302595e+18,2020-09-06T13:11:17.000Z,NaN,1302595225915985928,68,4,53,11,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True
498,2797521996,1.302552e+18,2020-09-06T10:20:31.000Z,NaN,1302552249919508480,414,26,231,144,NaN,...,2014-09-08T09:11:39.000Z,Conservative Member of Parliament for Haltempr...,2797521996,David Davis,188537,39,1333,1742,DavidDavisMP,True


In [ ]:
for col in list_of_cols_to_add:
    if col in df_tweets_i.columns:
        pass
    else:
        df_tweets_i[col] = "NA"

    #sort columns by alphabetic order
    col_list_df_tweets_i = df_tweets_i.columns.tolist()
    col_list_df_tweets_i.sort()
    df_tweets_i = df_tweets_i.reindex(columns=col_list_df_tweets_i)

            name = user_name + ".csv"
            path_for_table = os.path.join(dir_name_beta, name)
            if os.path.isfile(path_for_table) == False: #if this is the first table of tweets
                df_tweets_i.to_csv(path_for_table, index=True)
            else:
                df_tweets_i.to_csv(path_for_table, mode='a', index=True, header=False)
        except:
            print("no data / include in the json")
       
        with open(path_for_dir_retriving_tweets_streem, 'a') as f:
            print_stat = str(counter_loops) + " -> Got from twitter " + str(json_response["meta"]["result_count"]) + " tweets, and there are more tweets of that user to get, I am bringing more tweets!"
            f.write(print_stat)
            f.write('\n')
            print_total = "Total amount of tweets: " + str(num_of_returned_tweets)
            f.write(print_total)
            f.write('\n')
            f.write('\n')


In [ ]:
a = pd.json_normalize(json_response["data"])
b = pd.json_normalize(json_response["includes"], ["users"]).add_prefix("users.")
#c = pd.json_normalize(json_response["includes"]["places"]).add_prefix("places.")
df_tweets_i = pd.merge(a, b, left_on="author_id", right_on="users.id")
list_of_cols_to_add = ['author_id', 'conversation_id', "id", 'created_at','entities.mentions',
    'public_metrics.like_count', 'public_metrics.quote_count',
   'public_metrics.reply_count', 'public_metrics.retweet_count','referenced_tweets', 'text',
    'users.created_at', 'users.description','users.id', 'users.name',
   'users.public_metrics.followers_count', 'users.public_metrics.following_count',
   'users.public_metrics.listed_count', 'users.public_metrics.tweet_count',
   'users.username', 'users.verified']

list_cols_to_drop = [x for x in df_tweets_i.columns if x not in list_of_cols_to_add]

##droping labels we don't need
df_tweets_i = df_tweets_i.drop(labels=list_cols_to_drop, axis = 1, errors = "ignore")

{'data': {'source': 'Twitter for iPhone',
  'text': 'Tweet with location',
  'id': '1470753200311517189',
  'public_metrics': {'retweet_count': 0,
   'reply_count': 0,
   'like_count': 0,
   'quote_count': 0},
  'reply_settings': 'everyone',
  'author_id': '1460590577465147395',
  'conversation_id': '1470753200311517189',
  'geo': {'place_id': '07d9e57276484001'},
  'lang': 'en',
  'created_at': '2021-12-14T13:51:02.000Z'},
 'includes': {'users': [{'username': 'MichaelKobaiva1',
    'public_metrics': {'followers_count': 1,
     'following_count': 1,
     'tweet_count': 6,
     'listed_count': 0},
    'id': '1460590577465147395',
    'name': 'Michael Kobaivanov',
    'description': '',
    'verified': False,
    'created_at': '2021-11-16T12:48:51.000Z'}],
  'places': [{'country_code': 'IL',
    'id': '07d9e57276484001',
    'full_name': 'IDC Library - ספריית המרכז הבינתחומי',
    'country': 'Israel',
    'geo': {'type': 'Feature',
     'bbox': [34.83619828881944,
      32.17587479871207